In [4]:
!pip install transformers torch ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00


In [11]:
# Ensure that the necessary libraries are installed
!pip install transformers ipywidgets torch

import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import ipywidgets as widgets
from IPython.display import display



In [12]:
# Initialize GPT-2 tokenizer and model
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

In [13]:
# Function to generate song lyrics with only verses (no chorus)
def generate_song_verses(theme, emotion, num_verses, max_length=300):
    # Define a song-like prompt with only verses structure
    prompt = f"Theme: {theme}\nEmotion: {emotion}\nSong Lyrics:\n"

    song_verses = []

    for i in range(1, num_verses + 1):
        # Generate lyrics for the verse
        verse_prompt = f"{prompt}Verse {i}:\n"

        # Encode verse prompt and generate lyrics for the verse
        input_ids_verse = gpt2_tokenizer.encode(verse_prompt, return_tensors="pt")
        verse_output = gpt2_model.generate(
            input_ids_verse,
            max_length=max_length,
            num_return_sequences=1,
            no_repeat_ngram_size=2,
            temperature=0.7,
            top_k=50,
            top_p=0.85,
            do_sample=True,
            pad_token_id=gpt2_tokenizer.eos_token_id
        )

        verse_lyrics = gpt2_tokenizer.decode(verse_output[0], skip_special_tokens=True).strip()
        verse = verse_lyrics.split(f"Verse {i}:")[-1].strip()

        # Add the verse to the song lyrics list
        song_verses.append(f"Verse {i}:\n{verse}\n")

    return "\n".join(song_verses)

In [14]:
# Define interactive widgets for user input
theme_widget = widgets.Text(
    value='love',
    description='Theme:',
    placeholder='Enter the theme of the song'
)

emotion_widget = widgets.Text(
    value='happy',
    description='Emotion:',
    placeholder='Enter the emotion of the song'
)

num_verses_widget = widgets.IntSlider(
    value=3,
    min=1,
    max=10,
    step=1,
    description='Verses:',
    continuous_update=False
)

In [15]:
# Function to generate and display lyrics based on user input
def generate_lyrics_action(theme, emotion, num_verses):
    # Generate the song lyrics based on the theme, emotion, and number of verses
    lyrics = generate_song_verses(theme, emotion, num_verses)

    # Display generated lyrics
    output_widget.value = lyrics

In [16]:
# Create a button to trigger lyrics generation
generate_button = widgets.Button(description="Generate Lyrics")
generate_button.on_click(lambda x: generate_lyrics_action(theme_widget.value, emotion_widget.value, num_verses_widget.value))


In [17]:
# Output widget to display the generated lyrics
output_widget = widgets.Textarea(
    value='',
    description='Generated Lyrics:',
    layout=widgets.Layout(width='100%', height='300px'),
    disabled=True,
    placeholder="Generated lyrics will appear here..."
)

In [18]:
# Display all widgets and output
display(
    widgets.HTML("<h2>Song Lyrics Generator (using GPT-2)</h2>"),
    theme_widget,
    emotion_widget,
    num_verses_widget,
    generate_button,
    output_widget
)

HTML(value='<h2>Song Lyrics Generator (using GPT-2)</h2>')

Text(value='love', description='Theme:', placeholder='Enter the theme of the song')

Text(value='happy', description='Emotion:', placeholder='Enter the emotion of the song')

IntSlider(value=3, continuous_update=False, description='Verses:', max=10, min=1)

Button(description='Generate Lyrics', style=ButtonStyle())

Textarea(value='', description='Generated Lyrics:', disabled=True, layout=Layout(height='300px', width='100%')…